# Exemplo Regressão Logística

Neste exemplo iremos apresentar um modelo de regressão logística. O dataset utilizado será o dados_credito_small.csv.

O objetivo desta análise é desenvolver um modelo que buscará predizer a probabilidade de pagamento de um indivíduo.

O modelo a ser ajustado será: 

$$
\begin{equation}
P(pagar_i) = \frac{1}{1 + \mathrm{e}^{-(\beta_0 + \sum\beta_i caracteristicas_i)}}
\tag{1}
\end{equation}
$$

## Carregar pacotes

In [1]:
library(tidyverse)
library(magrittr)
library(caret)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift



## Carregar dados

In [2]:
dados_credito <- read_csv( file = "/home/vm-data-science/dados/dados_credito_small.csv" )
dados_credito %>% head

Parsed with column specification:
cols(
  id = col_integer(),
  idade = col_integer(),
  educacao = col_integer(),
  tipo_emprego = col_integer(),
  renda = col_integer(),
  divida = col_double(),
  outras_div = col_double(),
  default = col_character(),
  educacao_cat = col_character(),
  idade_cat = col_character(),
  renda_cat = col_character()
)


id,idade,educacao,tipo_emprego,renda,divida,outras_div,default,educacao_cat,idade_cat,renda_cat
1,41,3,18,176,10.3,5.01,nao_pagou,ensin_sup,acima_40,alta
2,27,1,10,31,17.3,4.00,pagou,ensin_basico,abaixo_30,baixa_media
3,40,1,15,55,5.5,2.17,pagou,ensin_basico,entre_30_40,alta
4,41,1,15,120,2.9,0.82,pagou,ensin_basico,acima_40,alta
5,24,2,2,28,17.3,3.06,nao_pagou,ensin_medio,abaixo_30,baixa_media
6,41,2,5,25,10.2,2.16,pagou,ensin_medio,acima_40,baixa_media


Pequeno ajuste necessário.

Também devemos criar um coluna nova, sendo 1 = pagou e 0 = não pagou.

In [3]:
dados_credito %<>%
    mutate( default_dummy = ifelse( default == 'pagou', 1, 0 ),
            default = as.factor(default) )

## Análises

Antes de iniciarmos a construção dos modelos é importante realizar um breve análise exploratória buscando comparar os indivíduos que pagaram e não pagaram.

### Análises em relação variáveis quantitativas

In [4]:
dados_credito %>% 
    select( -id, -educacao, -tipo_emprego, -default_dummy ) %>% 
    group_by( default ) %>% 
    summarise_if( is.numeric, mean )

default,idade,renda,divida,outras_div
nao_pagou,33.41085,44.61240,14.597674,4.106667
pagou,35.15633,45.64151,8.698383,2.647197


### Análises em relação as variáveis categóricas

In [5]:
dados_credito %>% 
    group_by( default ) %>% 
    count( educacao_cat ) %>% 
    spread( key = default, value = n )

educacao_cat,nao_pagou,pagou
ensin_basico,55,214
ensin_medio,41,98
ensin_sup,33,59


In [6]:
dados_credito %>% 
    group_by( default ) %>% 
    count( idade_cat ) %>% 
    spread( key = default, value = n )

idade_cat,nao_pagou,pagou
abaixo_30,57,119
acima_40,27,89
entre_30_40,45,163


In [7]:
dados_credito %>% 
    group_by( default ) %>% 
    count( renda_cat ) %>% 
    spread( key = default, value = n )

renda_cat,nao_pagou,pagou
alta,28,96
baixa,49,77
baixa_media,24,103
media_alta,28,95


### Divisão das amostras treinamento e teste

In [8]:
# amostra de treino
set.seed(12345)
dados_credito_train <- dados_credito %>% 
                            sample_frac(., 0.7)

In [9]:
# amostra de teste
dados_credito_test <- setdiff( dados_credito, dados_credito_train )

In [10]:
print( paste('o banco de dados tem ao todo', dim(dados_credito)[1], 'linhas e', dim(dados_credito)[2], 'colunas') )
print( paste('a amostra de treino tem', dim(dados_credito_train)[1], 'linhas e', dim(dados_credito_train)[2], 'colunas') )
print( paste('a amostra de teste tem', dim(dados_credito_test)[1], 'linhas e', dim(dados_credito_test)[2], 'colunas') ) 

[1] "o banco de dados tem ao todo 500 linhas e 12 colunas"
[1] "a amostra de treino tem 350 linhas e 12 colunas"
[1] "a amostra de teste tem 150 linhas e 12 colunas"


### Treinamento modelo de regressão logística

- modelo 1: modelo com todas as variáveis

In [11]:
model_full <- glm( formula = default_dummy ~ .,
                   family = 'binomial',
                   data = dados_credito_train %>% select( -id, -default) )
summary(model_full)


Call:
glm(formula = default_dummy ~ ., family = "binomial", data = dados_credito_train %>% 
    select(-id, -default))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.6382  -0.4022   0.3375   0.6468   1.8976  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              2.333333   1.573712   1.483   0.1382    
idade                   -0.015337   0.046279  -0.331   0.7403    
educacao                 0.528333   0.586199   0.901   0.3674    
tipo_emprego             0.187392   0.039979   4.687 2.77e-06 ***
renda                   -0.018243   0.007839  -2.327   0.0200 *  
divida                  -0.183874   0.035751  -5.143 2.70e-07 ***
outras_div               0.085518   0.093034   0.919   0.3580    
educacao_catensin_medio -0.785871   0.683491  -1.150   0.2502    
educacao_catensin_sup   -1.321828   1.412456  -0.936   0.3494    
idade_catacima_40       -0.440251   0.987109  -0.446   0.6556    
idade_catentre_30_40    -

Escreva a equação do modelo:

- modelo 2: O seu modelo

### Teste dos modelos de regressão logística

Para os problemas de regressão logística, iremos utilizar a métrica da Acurácia para validação dos modelos.

- Gera as previsões. (lembre-se de adicionar o seu modelo)

In [12]:
dados_credito_test %<>% 
    mutate( previsao_model_full = predict( model_full, ., type = 'response' ) )

- O modelo gera probabilidades.

In [13]:
dados_credito_test %>% 
    select( default_dummy, previsao_model_full ) %>% 
    head

default_dummy,previsao_model_full
1,0.8892047
1,0.8161954
1,0.9493347
1,0.8435584
1,0.7879008
0,0.8707278


Transforma as probabilidades em classificações. (lembre-se de adicionar o seu modelo)

Neste exemplo, vamos assumir que valores acima de 0.5 como 'pagou' e igual ou abaixo de 0.5 como 'não_pagou'.

In [26]:
dados_credito_test %<>% 
    mutate( previsao_model_full_bin = as.factor( ifelse( previsao_model_full > 0.55, 'pagou', 'nao_pagou') ) )

In [27]:
dados_credito_test %>% 
    select( default, previsao_model_full_bin ) %>% 
    head

default,previsao_model_full_bin
pagou,pagou
pagou,pagou
pagou,pagou
pagou,pagou
pagou,pagou
nao_pagou,pagou


In [28]:
dados_credito_test %>% 
    count( previsao_model_full_bin )

previsao_model_full_bin,n
nao_pagou,27
pagou,123


- Matriz de confusão e Acurácia do modelo 1.

In [29]:
confusionMatrix(dados_credito_test$default, 
                dados_credito_test$previsao_model_full_bin,
                positive = 'pagou')

Confusion Matrix and Statistics

           Reference
Prediction  nao_pagou pagou
  nao_pagou        16    20
  pagou            11   103
                                          
               Accuracy : 0.7933          
                 95% CI : (0.7197, 0.8551)
    No Information Rate : 0.82            
    P-Value [Acc > NIR] : 0.8312          
                                          
                  Kappa : 0.3805          
 Mcnemar's Test P-Value : 0.1508          
                                          
            Sensitivity : 0.8374          
            Specificity : 0.5926          
         Pos Pred Value : 0.9035          
         Neg Pred Value : 0.4444          
             Prevalence : 0.8200          
         Detection Rate : 0.6867          
   Detection Prevalence : 0.7600          
      Balanced Accuracy : 0.7150          
                                          
       'Positive' Class : pagou           
                                          

Matriz de confusão do modelo 2.

Você conseguiu criar um modelo melhor?